In [2]:
import tensorflow.keras as keras
from keras.datasets import cifar10
from keras.layers import Input, DepthwiseConv2D, Conv2D
from keras.models import Model

# Load dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

# Define the Depthwise Separable Convolution Layer
class DepthwiseSeparableConv2D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding, activation):
        super(DepthwiseSeparableConv2D, self).__init__()
        # Depthwise convolution
        self.depthwise = DepthwiseConv2D(kernel_size=kernel_size, padding=padding, activation=activation)
        # Pointwise convolution
        self.pointwise = Conv2D(filters=filters, kernel_size=(1, 1), activation=activation)

    def call(self, input_tensor):
        # Apply depthwise convolution first, followed by pointwise convolution
        x = self.depthwise(input_tensor)
        return self.pointwise(x)

# Constructing a model using the Depthwise Separable Convolution Layer
visible = Input(shape=(32, 32, 3))  # CIFAR-10 images have size 32x32 with 3 color channels (RGB)
depthwise_separable = DepthwiseSeparableConv2D(filters=64, kernel_size=(3,3), padding="valid", activation="relu")(visible)
depthwise_model = Model(inputs=visible, outputs=depthwise_separable)

# Print model summary to see the number of parameters
depthwise_model.summary()

# Comparison with a similar model using a regular Conv2D layer
normal = Conv2D(filters=64, kernel_size=(3,3), padding="valid", activation="relu")(visible)
normal_model = Model(inputs=visible, outputs=normal)

# Print model summary of the normal Conv2D model
normal_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_separable_conv2d_1         │ (None, 30, 30, 64)          │             286 │
│ (DepthwiseSeparableConv2D)           │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 286 (1.12 KB)

 Trainable params: 286 (1.12 KB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 30, 30, 64)          │           1,792 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,792 (7.00 KB)

 Trainable params: 1,792 (7.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow.keras as keras
from keras.datasets import cifar10
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, SeparableConv2D
from keras.models import Model
import tensorflow as tf

# Load CIFAR-10 dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

# Normalize data to range [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# VGG block with standard convolutional layers
def vgg_block(layer_in, n_filters, n_conv):
    for _ in range(n_conv):
        layer_in = Conv2D(filters=n_filters, kernel_size=(3, 3), padding="same", activation="relu")(layer_in)
    layer_in = MaxPooling2D((2, 2), strides=(2, 2))(layer_in)
    return layer_in

# VGG block with depthwise separable convolutional layers
def vgg_depthwise_block(layer_in, n_filters, n_conv):
    for _ in range(n_conv):
        layer_in = SeparableConv2D(filters=n_filters, kernel_size=(3, 3), padding="same", activation="relu")(layer_in)
    layer_in = MaxPooling2D((2, 2), strides=(2, 2))(layer_in)
    return layer_in

# Model with standard convolutional layers
def create_standard_vgg():
    visible = Input(shape=(32, 32, 3))
    layer = vgg_block(visible, 64, 2)
    layer = vgg_block(layer, 128, 2)
    layer = vgg_block(layer, 256, 2)
    layer = Flatten()(layer)
    output = Dense(units=10, activation="softmax")(layer)
    model = Model(inputs=visible, outputs=output)
    return model

# Model with depthwise separable convolutional layers
def create_depthwise_vgg():
    visible = Input(shape=(32, 32, 3))
    layer = vgg_depthwise_block(visible, 64, 2)
    layer = vgg_depthwise_block(layer, 128, 2)
    layer = vgg_depthwise_block(layer, 256, 2)
    layer = Flatten()(layer)
    output = Dense(units=10, activation="softmax")(layer)
    model = Model(inputs=visible, outputs=output)
    return model

# Create and compile standard VGG model
standard_vgg = create_standard_vgg()
standard_vgg.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

# Train standard VGG model
print("Training standard VGG model...")
history_standard = standard_vgg.fit(trainX, trainY, batch_size=128, epochs=10, validation_data=(testX, testY))

# Evaluate standard VGG model
print("\nEvaluating standard VGG model...")
standard_loss, standard_acc = standard_vgg.evaluate(testX, testY)
print(f"Standard VGG Model Accuracy: {standard_acc:.4f}")

# Create and compile depthwise separable VGG model
depthwise_vgg = create_depthwise_vgg()
depthwise_vgg.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

# Train depthwise separable VGG model
print("\nTraining depthwise separable VGG model...")
history_depthwise = depthwise_vgg.fit(trainX, trainY, batch_size=128, epochs=10, validation_data=(testX, testY))

# Evaluate depthwise separable VGG model
print("\nEvaluating depthwise separable VGG model...")
depthwise_loss, depthwise_acc = depthwise_vgg.evaluate(testX, testY)
print(f"Depthwise Separable VGG Model Accuracy: {depthwise_acc:.4f}")

# Compare model parameter counts
print("\nModel Summary - Standard VGG:")
standard_vgg.summary()

print("\nModel Summary - Depthwise Separable VGG:")
depthwise_vgg.summary()


Training standard VGG model...
Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 1208s 3s/step - accuracy: 0.3288 - loss: 1.8092 - val_accuracy: 0.6159 - val_loss: 1.0965
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 1226s 3s/step - accuracy: 0.6396 - loss: 1.0152 - val_accuracy: 0.7029 - val_loss: 0.8536
Epoch 3/10
 98/391 ━━━━━━━━━━━━━━━━━━━━ 13:57 3s/step - accuracy: 0.7343 - loss: 0.7681

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import decode_predictions

# CIFAR-10 class names
cifar10_classes = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

# Function to preprocess an uploaded image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(32, 32))  # Resize to 32x32
    img_array = img_to_array(img)
    img_array = img_array.astype("float32") / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict and display results
def predict_image(model, image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]
    return cifar10_classes[predicted_class]

# Upload images
uploaded_files = files.upload()  # Use Colab to upload images

# Test the uploaded images on both models
for filename in uploaded_files:
    print(f"\nTesting image: {filename}")
    image_path = filename
    plt.imshow(load_img(image_path))
    plt.axis("off")
    plt.show()

    # Predict using standard VGG model
    result_standard = predict_image(standard_vgg, image_path)
    print(f"Standard VGG Model Prediction: {result_standard}")

    # Predict using depthwise separable VGG model
    result_depthwise = predict_image(depthwise_vgg, image_path)
    print(f"Depthwise Separable VGG Model Prediction: {result_depthwise}")
